![Explanation for you <3](https://s3.us-east-2.amazonaws.com/journeynorth.org/images/graphics/mclass/Lat_Long.gif)

In [1]:
import json
import os
from matplotlib import pyplot as plt
import pandas as pd


In [2]:

# open all.json Update the image list from nasa
# os.system("wget -r https://epic.gsfc.nasa.gov/api/enhanced/all")
# os.rename("epic.gsfc.nasa.gov/api/enhanced/all", "epic.gsfc.nasa.gov/api/enhanced/all.json")


In [ ]:

def contruct_date_list(): 
    dates = []
    with open("epic.gsfc.nasa.gov/api/enhanced/all.json") as f:
        data = json.load(f)
        for d in data:
            dates.append(d["date"])
    return dates

dates = contruct_date_list()
            
image_folders = []
# construct all image folder urls
for d in dates:
    image_folders.append(f"https://epic.gsfc.nasa.gov/api/enhanced/date/{d}")
print(len(image_folders))
print(len(dates))
print(image_folders[0])


In [ ]:
# PLEASE REMOVE DUPLICATES !!!!!111!!!11!1!1! Danger Below
# make all image folders
''' os.system("mkdir images")
for i in range(len(dates)):
    os.system(f"mkdir images/{dates[i]}")
    os.system(f"wget -P images/{dates[i]} {image_folders[i]}")
    os.system(f"mv images/{dates[i]}/{dates[i]} images/{dates[i]}/{dates[i]}.json") '''


In [5]:
folders = os.listdir("images")
folders.sort()
if ".DS_Store" in folders:
    folders.remove(".DS_Store")
len(folders)
for i in range(len(folders)):
    os.system(f"cp images/{folders[i]}/{folders[i]}.json combined")


In [6]:
files = os.listdir("combined")
if ".DS_Store" in files:
    files.remove(".DS_Store")

In [7]:
lat_coordinates = []
lon_coordinates = []
for i in range(len(files)):
    with open(f"combined/{files[i]}") as f:
        data = json.load(f)
        for p in range(len(data)):
            lat_coordinates.append(data[p]["centroid_coordinates"]["lat"])
            lon_coordinates.append(data[p]["centroid_coordinates"]["lon"])

        

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np

# set up orthographic map projection with
# perspective of satellite looking down at 45N, 100W.
# use low resolution coastlines.
map = Basemap(projection='ortho', lat_0=30, lon_0=10, resolution='c')

# draw coastlines, country boundaries, fill continents.
map.drawcoastlines(linewidth=0.5)
map.drawcountries(linewidth=0.25)
map.fillcontinents(color='grey', lake_color='aqua')

# draw the edge of the map projection region (the projection limb)
map.drawmapboundary(fill_color='aqua')

# draw lat/lon grid lines every 30 degrees.
map.drawmeridians(np.arange(0, 360, 30))
map.drawparallels(np.arange(-90, 90, 30))

# Plot lon and lat coordinates on the map. Somehow its lon first and then lat
x, y = map(lon_coordinates, lat_coordinates)
map.scatter(x, y, marker='D', color='m', label="possible centroid coordinates")

p, q = map(12.916667, 50.833332)
map.scatter(p, q, marker="o", color="b")

# Add label for the point p, q
plt.text(p, q, "Chemnitz", fontsize=12, ha='left', va='bottom', color='white', weight="bold")

# plot labels
plt.title("Epic Satellite Image Centroid Coordinates")
plt.legend()
plt.show()


In [ ]:
df_lat = pd.DataFrame(lat_coordinates)
df_lon = pd.DataFrame(lon_coordinates)
print("lat describe: ")
print(df_lat.describe())
print("---------------")
print("lon describe: ")
print(df_lon.describe())

In [ ]:
# make distribution of lat and lon
plt.hist(lon_coordinates, bins=100, label="longitude in ° (Width 0° = greenwich)")
plt.ylabel("Frequency")
plt.hist(lat_coordinates, bins=100, label="latitude in ° (Height above equator)")
plt.ylabel("Frequency")
plt.legend()
plt.show()


In [2]:
import time
import datetime
import json
import os
import random
import numpy as np
import webbrowser
from PIL import Image
import matplotlib.pyplot as plt
from rich.progress import track
import subprocess
import pandas as pd
import torch
import torch.nn as nn
import torchvision
from torchvision.utils import save_image, make_grid
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter

In [ ]:
for i in range(len(files)):
    with open(f"combined/{files[i]}") as f:
        data = json.load(f)
        for p in range(len(data)):
            # print(data[p]["image"] + ".png")
            lat_coordinates.append(data[p]["centroid_coordinates"]["lat"])
            lon_coordinates.append(data[p]["centroid_coordinates"]["lon"])
        for j in range(len(data)):
            image_name = data[j]["image"]
            lat = data[j]["centroid_coordinates"]["lat"]
            lon = data[j]["centroid_coordinates"]["lon"]
            new_image_name = f"{image_name}_{lat}_{lon}.png"
            os.system(f"cp download/earth/{image_name}.png pics_with_coords/{new_image_name}")

In [3]:
from LocationRegressor import LocationRegressor

trans = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.Grayscale(),
        transforms.ToTensor()
    ])

model = LocationRegressor()
summary(model, input_size=(1, 1, 64, 64))

Layer (type:depth-idx)                   Output Shape              Param #
LocationRegressor                        [1, 2]                    --
├─Sequential: 1-1                        [1, 2]                    --
│    └─Conv2d: 2-1                       [1, 64, 64, 64]           640
│    └─MaxPool2d: 2-2                    [1, 64, 16, 16]           --
│    └─ReLU: 2-3                         [1, 64, 16, 16]           --
│    └─Conv2d: 2-4                       [1, 128, 16, 16]          73,856
│    └─MaxPool2d: 2-5                    [1, 128, 4, 4]            --
│    └─ReLU: 2-6                         [1, 128, 4, 4]            --
│    └─Conv2d: 2-7                       [1, 256, 4, 4]            295,168
│    └─MaxPool2d: 2-8                    [1, 256, 1, 1]            --
│    └─ReLU: 2-9                         [1, 256, 1, 1]            --
│    └─Flatten: 2-10                     [1, 256]                  --
│    └─Linear: 2-11                      [1, 2]                    514
Tota

In [5]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

dataset = ImageFolder(root="pics_with_coords", transform=trans)